IMPORTING THE LIBRARIES

In [26]:
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import time
import torch
import torchvision.models as models
import torch.quantization

In [27]:
# !pip install torch torchvision -f https://storage.googleapis.com/tpu-pytorch/wheels/colab.html


In [28]:
# import torch
# import torch_xla
# import torch_xla.core.xla_model as xm

# # Check TPU device
# device = xm.xla_device()
# print(f"Using TPU: {device}")

# # Run a test operation
# x = torch.tensor([1, 2, 3], device=device)
# print(f"Tensor on TPU: {x}")


In [29]:
# import torch
# import torch_xla.core.xla_model as xm

# device = xm.xla_device()
# print(f"Using TPU: {device}")

# x = torch.tensor([1, 2, 3], device=device)
# print(f"Tensor on TPU: {x}")


In [30]:
# from google.colab import drive
# drive.mount('/content/drive')

In [31]:
import os

folder_path = r'C:\Users\yagni\PythonProjects\Pathpal\Obstacle Detection\City_dataset_full'

if os.path.exists(folder_path) and os.path.isdir(folder_path):
    print("The folder exists.")
else:
    print("The folder does not exist.")


The folder does not exist.


In [32]:
import torch.nn as nn

In [33]:
device = torch.device('cuda')

In [34]:
train_images = []
Masks = []

In [35]:
from torchvision import transforms

In [36]:
print(torch.cuda.is_available())

True


In [37]:

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),# Converts the image to a tensor and scales pixel values to [0, 1]
])


In [38]:
masks_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x[:1, :, :]),
    transforms.Lambda(lambda x : x > 0.04),# Converts the image to a tensor and scales pixel values to [0, 1]
])

In [39]:
mask_transform_india= transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
      transforms.Lambda(lambda x: x[:1, :, :]),
    transforms.Lambda(lambda x : x > 0.5),# Converts the image to a tensor and scales pixel values to [0, 1]
])

In [40]:
k=0
for dirpath, dirnames, filenames in os.walk(r"C:\Users\yagni\PythonProjects\Pathpal\Obstacle Detection\Indian_road_data\Indian_road_data\Raw_images\train"):
    if k==0:
        k=k+1
        pass
    elif k!=0:
        items = os.listdir(dirpath)
        for item in items:
            img = Image.open(os.path.join(dirpath,item))
            train_images.append(transform(img))

In [41]:
print(len(train_images))

0


In [42]:
j=0
for dirpath, dirnames, filenames in os.walk(r"C:\Users\yagni\PythonProjects\Pathpal\Obstacle Detection\Indian_road_data\Indian_road_data\Masks\train"):
    if j==0:
        j=j+1
        pass
    elif j!=0:
        masks = os.listdir(dirpath)
        for mask in masks:
            if not mask.endswith('.png'):
                continue
            img = Image.open(os.path.join(dirpath,mask))
            Masks.append(mask_transform_india(img))

In [43]:
print(len(Masks))

0


In [44]:
i=0
for dirpath, dirnames, filenames in os.walk(r"C:\Users\yagni\PythonProjects\Pathpal\Obstacle Detection\City_dataset_full"):
    if dirnames and i==0 :
        i=i+1
        pass

    elif dirnames and i!=0 :
        path = os.path.join(dirpath,"images")
        path_masks= os.path.join(dirpath,"mask")
        items = os.listdir(path)
        masks = os.listdir(path_masks)
        count = 0
        for item in items:
            img = Image.open(os.path.join(path,item))
            train_images.append(transform(img))

        for mask in masks:
            if count % 3 == 1 :
                msk = os.path.join(path_masks,mask)
                msk = Image.open(msk)
                Masks.append(masks_transform(msk))
                count =count +1
            else :
                count= count +1

In [47]:
print(train_images[0].shape)
print(Masks[0].shape)

IndexError: list index out of range

In [46]:
# torch.tensor(train_images,dtype=torch.float32) this method is wrong u cant convert a list of tensors to a tensor u need to use stack
X_train = torch.stack(train_images, dim =0)
y_train = torch.stack(Masks, dim=0)

RuntimeError: stack expects a non-empty TensorList

In [ ]:
perm = torch.randperm(X_train.size(0))

In [ ]:
X_train_shuffled = X_train[perm]
y_train_shuffled = y_train[perm]

In [ ]:
y_train.squeeze(dim=1).size()

torch.Size([3688, 256, 256])

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, images, masks):
        self.images = images
        self.masks = masks

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        mask = self.masks[idx]
        return image, mask


In [ ]:
from torch.utils.data import random_split


In [ ]:
total_size = len(CustomDataset(X_train,y_train))
train_size = int(0.9 * total_size)  # 90% for training
val_size = total_size - train_size   # The remaining 10% for validation


In [ ]:
train_dataset, val_dataset = random_split(CustomDataset(X_train,y_train), [train_size, val_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size= 8 ,shuffle=False)

In [ ]:
def conv_block(input_channels,output_channels):
   block = nn.Sequential(
   nn.Conv2d(in_channels=input_channels,out_channels = output_channels,kernel_size=3,stride=1,padding=1),
   nn.ReLU(),
   nn.Conv2d(in_channels=output_channels,out_channels = output_channels,kernel_size=3,stride=1,padding=1),
   nn.ReLU()
   )
   return block

In [ ]:
class Encoder_Block(nn.Module):
  def __init__(self,input_channels,output_channels):
    super().__init__()
    self.conv = conv_block(input_channels,output_channels)
    self.maxpool = nn.MaxPool2d(kernel_size=2,stride=2)

  def forward(self,x):
    x = self.conv(x)
    p = self.maxpool(x)
    return x ,p

In [ ]:
class Decoder_block(nn.Module):

  def __init__(self,input_channels,output_channels):
    super().__init__()
    self.conv_transpose = nn.ConvTranspose2d(input_channels,output_channels,kernel_size=2,stride=2,padding=0)
    self.conv = conv_block(input_channels,output_channels)

  def forward(self,x,skip_features):
    x = self.conv_transpose(x)
    x = torch.cat((x,skip_features),dim=1)
    x = self.conv(x)
    return x

In [ ]:
class Encoder_block(nn.Module):

  def __init__(self,input_channels,output_channels):
    super().__init__()
    self.conv = conv_block(input_channels,output_channels)
    self.maxpool = nn.MaxPool2d(kernel_size=2,stride=2)

  def forward(self,x):
    x = self.conv(x)
    p = self.maxpool(x)
    return x ,p

In [ ]:
class UNET(nn.Module):
 def __init__(self):
    super().__init__()
    self.encoder1 = Encoder_block(3,64)
    self.encoder2 = Encoder_block(64,128)
    self.encoder3 = Encoder_block(128,256)
    self.encoder4 = Encoder_block(256,512)

    self.base_layer = conv_block(512,1024)

    self.decoder1 = Decoder_block(1024,512)
    self.decoder2 = Decoder_block(512,256)
    self.decoder3 = Decoder_block(256,128)
    self.decoder4 = Decoder_block(128,64)

    self.final_layer =nn.Sequential(nn.Conv2d(in_channels=64,out_channels=1,kernel_size=1), nn.Sigmoid())

 def forward(self,X):

      x1, p1 = self.encoder1(X)

      x2, p2 = self.encoder2(p1)

      x3, p3 = self.encoder3(p2)

      x4, p4 = self.encoder4(p3)

      base = self.base_layer(p4)

      dec1 = self.decoder1(base,x4)

      dec2 = self.decoder2(dec1,x3)

      dec3 = self.decoder3(dec2,x2)

      dec4 = self.decoder4(dec3,x1)

      final_output = self.final_layer(dec4)
      return final_output

In [ ]:
model_testing = UNET().to(device)

In [ ]:
# model_testing.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
# model_testing = torch.quantization.prepare_qat(model_testing)


In [ ]:
epochs = 20

In [ ]:
# loss_func =  nn.BCELoss()
optimizer  = torch.optim.Adam(model_testing.parameters(),lr=0.001)

In [ ]:
Total_train = (len(train_loader.dataset)*256*256)

In [ ]:
Total_val = len(val_loader.dataset)*256*256

In [ ]:
from tqdm import tqdm

In [ ]:
def dice_calc(prediction_set , truth_set):
  accumulator_dice = 0.0
  for pred , truth in zip(prediction_set,truth_set):
    # A = (pred==1).int().sum()
    A = torch.sum(pred==1)
    B = torch.sum(truth==1)
    C = torch.sum(pred*truth)
    accumulator_dice += (C*2/(A+B))
  return accumulator_dice/len(prediction_set)

#DICE_LOSS


In [ ]:
def dice_loss(prediction_set , truth_set):
  accumulator_dice = torch.tensor(0.0,device =device)
  for pred , truth in zip(prediction_set,truth_set):
    A = torch.sum(pred==1).float()
    B = torch.sum(truth==1).float()
    C = torch.sum(pred*truth).float()
    accumulator_dice += (C*2/(A+B))
  return 1 - accumulator_dice/len(prediction_set)

tested dice loss its working

In [ ]:
pred = torch.randn(4, 1, 128, 128, requires_grad=True)  # Predictions from the model
truth = torch.randint(0, 2, (4, 1, 128, 128)).float()  # Ground truth

# Compute Dice loss
loss = dice_calc(pred, truth)

print(loss.requires_grad)  # Should print True if the loss is part of the computational graph

loss.backward()  # Backpropagation should work without errors


True


In [ ]:
for epoch in tqdm(range(1,epochs+1)):

  model_testing.train()

  total_loss = 0.0

  dice_total = 0.0
  for batch_idx, (data,targets) in enumerate(train_loader):

    data , targets = data.to(device), targets.to(device).float()

    output = model_testing(data)

    loss = dice_loss(output ,targets)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    total_loss += loss.item()

    pred = (output > 0.5).float()

    dice_total += dice_calc(pred,data)

  dice_coefficient = dice_total/len(train_loader.dataset)
  print(f'Epoch: {epoch}, Loss: {total_loss/len(train_loader.dataset)}, dice_coefficient: {dice_coefficient}')

  model_testing.eval()

  val_loss = 0.0

  dice_val_total = 0

  correct_predictions = 0

  with torch.no_grad():

    for batch_idx, (data,targets) in enumerate(val_loader):

      data , targets = data.to(device), targets.to(device)

      val_output = model_testing(data)

      # val_output = val_output.squeeze(dim=1)
      loss = dice_loss(val_output,targets.float())

      val_loss += loss.item()

      pred = (val_output > 0.5)

      dice_val_total += dice_calc(pred,data)

    dice_coefficient_val = dice_val_total/len(val_loader.dataset)

    print(f'Validation Loss: {val_loss/len(val_loader.dataset)},  Dice Validation {dice_coefficient_val}')




  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1, Loss: 0.05621312243926234, dice_coefficient: 0.3300883173942566


  5%|▌         | 1/20 [02:09<40:54, 129.17s/it]

Validation Loss: 0.057517673749587724,  Dice Validation 0.32871556282043457
Epoch: 2, Loss: 0.05699332283361377, dice_coefficient: 0.3300895690917969


 10%|█         | 2/20 [04:15<38:16, 127.59s/it]

...

Validation Loss: 0.057517673749587724,  Dice Validation 0.32871556282043457
Epoch: 19, Loss: 0.05698838759057671, dice_coefficient: 0.33009985089302063


 95%|█████████▌| 19/20 [43:56<02:19, 139.78s/it]

Validation Loss: 0.057517673749587724,  Dice Validation 0.32871556282043457
Epoch: 20, Loss: 0.05699246087394662, dice_coefficient: 0.33008500933647156


100%|██████████| 20/20 [46:16<00:00, 138.80s/it]

Validation Loss: 0.057517673749587724,  Dice Validation 0.32871556282043457


In [ ]:

# BCE Loss Function
# def bce_loss(prediction_set, truth_set):
#     """
#     Compute Binary Cross-Entropy loss for a batch of predictions and truths.

#     Args:
#         prediction_set (torch.Tensor): Predicted logits or probabilities.
#         truth_set (torch.Tensor): Ground truth binary labels (0 or 1).

#     Returns:
#         torch.Tensor: Average BCE loss for the batch.
#     """
#     accumulator_bce = 0.0
#     for pred, truth in zip(prediction_set, truth_set):
#         # Apply sigmoid if `pred` contains raw logits
#         pred = torch.sigmoid(pred)
#         bce = F.binary_cross_entropy(pred, truth)
#         accumulator_bce += bce
#     return accumulator_bce / len(prediction_set)
loss_func = nn.BCE()
# Dice Coefficient Calculation
def dice_calc(prediction_set, truth_set):
    """
    Compute the Dice coefficient for predictions and ground truth.

    Args:
        prediction_set (torch.Tensor): Predicted binary values (0 or 1).
        truth_set (torch.Tensor): Ground truth binary values (0 or 1).

    Returns:
        torch.Tensor: Average Dice coefficient for the batch.
    """
    accumulator_dice = 0.0
    for pred, truth in zip(prediction_set, truth_set):
        A = torch.sum(pred == 1)
        B = torch.sum(truth == 1)
        C = torch.sum(pred * truth)
        accumulator_dice += ((C * 2) / (A + B))
    return accumulator_dice / len(prediction_set)

# Training Loop with BCE Loss
for epoch in tqdm(range(1, epochs + 1)):

    model_testing.train()

    total_loss = 0.0
    dice_total = 0.0

    for batch_idx, (data, targets) in enumerate(train_loader):
        data, targets = data.to(device), targets.to(device).float()

        output = model_testing(data)  # Raw logits

        loss = loss_func(output, targets)  # Compute BCE loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        pred = (output > 0.5).float()  # Convert logits to binary
        dice_total += dice_calc(pred, targets)

    dice_coefficient = dice_total / len(train_loader.dataset)
    print(f'Epoch: {epoch}, Loss: {total_loss / len(train_loader.dataset)}, Dice Coefficient: {dice_coefficient}')

    # Validation
    model_testing.eval()
    val_loss = 0.0
    dice_val_total = 0

    with torch.no_grad():
        for batch_idx, (data, targets) in enumerate(val_loader):
            data, targets = data.to(device), targets.to(device).float()

            val_output = model_testing(data)

            loss = loss_func(val_output, targets)  # Compute BCE loss for validation
            val_loss += loss.item()

            pred = (val_output > 0.5).float()  # Convert logits to binary
            dice_val_total += dice_calc(pred, targets)

        dice_coefficient_val = dice_val_total / len(val_loader.dataset)
        print(f'Validation Loss: {val_loss / len(val_loader.dataset)}, Dice Validation: {dice_coefficient_val}')


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1, Loss: 0.08676837773953766, Dice Coefficient: 0.0008464526035822928


  5%|▌         | 1/20 [03:43<1:10:38, 223.07s/it]

Validation Loss: 0.08828708489089801, Dice Validation: 0.0
Epoch: 2, Loss: 0.08666954779703957, Dice Coefficient: 0.0


 10%|█         | 2/20 [07:42<1:09:44, 232.50s/it]

...

Validation Loss: 0.08828708489089801, Dice Validation: 0.0
Epoch: 4, Loss: 0.08666954779703957, Dice Coefficient: 0.0


 20%|██        | 4/20 [15:46<1:03:45, 239.10s/it]

Validation Loss: 0.08828708489089801, Dice Validation: 0.0
Epoch: 5, Loss: 0.08666954779703957, Dice Coefficient: 0.0


 25%|██▌       | 5/20 [19:42<59:33, 238.22s/it]  

Validation Loss: 0.08828708489089801, Dice Validation: 0.0


In [ ]:
torch.save(model_testing.state_dict(), "weights.pth")

In [ ]:
# quantized_model = torch.quantization.convert(model_testing)

# # Save the quantized model
# torch.save(quantized_model, "qat_quantized_model.pth")

In [ ]:
# from google.colab import files
# files.download('runs/detect/train/weights/last.pt')


In [ ]:
Z